In [ ]:
import re, subprocess, pyodbc 
import numpy as np # linear algebra 
from sqlalchemy import func, create_engine
from config import connstr  
import pandas as pd

In [ ]:
engine = create_engine(connstr)
conn = engine.connect()
query = """ \
    SELECT FP.ASPX , FP.PK_FormPage,  FP.Text, MCK.* FROM vw_MetricsCompositeKeys MCK 
    INNER JOIN fsma_FormPages FP ON FP.PK_FormPage=MCK.FK_FormPage
    WHERE PK_FORM = '2021-A-EINSTEIN' AND PK_FormPage <> 2072
""" 
df = pd.read_sql(query,con=conn)  
df
alt = ''
for i,r in df.iterrows():
    text = re.sub('Section \d:' , f'Section {i+1}:' , r['Text'])
    alt = alt + f"""
    UPDATE fsma_FormPages 
    SET Text = '{text}', sortpos = {i+1}, ASPX = 'EINSTEIN/2021/2021_A_EINSTEIN_{i}.aspx'
    WHERE PK_FormPage = {r['PK_FormPage']}
    """
print( f'{alt}' )

In [ ]:
secs = [
      {'s':1, 'name':'EinsteinPublicIP'}
    , {'s':2, 'name':'EinsteinUnannounced'}
    , {'s':3, 'name':'EinsteinBGP'}
    , {'s':4, 'name':'EinsteinUnmonitoredTraffic'}
    , {'s':5, 'name':'EinsteinTAPS'} 
    , {'s':7, 'name':'EinsteinNetChannelsDS'}
    , {'s':8, 'name':'EinsteinNAT'}
    ]

In [ ]:

lst = [] 
for d in secs:
    df = pd.read_csv(f"C:\\Users\\Tim\\Documents\\EINSTEIN\\ss\\{d['s']}.csv")
    df['SECTION'] = d['s']
    df['TABLE_NAME'] = d['name']
    df['COLUMN_NAME'] = df['Column Headings'].apply(lambda s: re.sub('[^A-Za-z0-9_]','',s))
    df = df.rename(columns={'Column Headings': 'Headings'
                            , 'Enumerated List': 'Enumerate'
                            , 'Data Entry Validations': 'Validations'
                            , 'Additional Help Text': 'Help'})
    lst.append(df)
df = pd.concat(lst)
df 

In [ ]:
alt = ''
for i,r in df.iterrows():
    alt = alt + f"""
    IF NOT EXISTS(SELECT * FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME='{r['TABLE_NAME']}' and COLUMN_NAME='{r['COLUMN_NAME']}')
    BEGIN
        ALTER TABLE {r['TABLE_NAME']} ADD {r['COLUMN_NAME']} NVARCHAR(255) NULL
    END
    GO
    """
print( f'{alt}' )